# HW10
## Problem 1: Computational cost

Input layer:  $O(d\cdot k)$  
Hidden layer: $ O(k\cdot k \cdot (q-1))$  
Output layer: $ O(k)$  

**In total: $O(d\cdot k + (q-1)\cdot k^2) + k) = O(d\cdot k + (q-1)\cdot k^2)) $**


## Problem 2: Neural networks with PyTorch

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

Load CIFAR10 train and test set

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

100%|██████████| 170498071/170498071 [00:03<00:00, 47763214.95it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


1. Reducing the size of the training and test accordingly

In [3]:
# car: 1
# dog: 5
# ship: 8
classes = {'car', 'dog', 'ship'}
class_mapping={1:0,5:1,8:2}

trainset = [ex for ex in trainset if ex[1]==1 or ex[1]==5 or ex[1]==8]
testset = [ex for ex in testset if ex[1]==1 or ex[1]==5 or ex[1]==8]

trainloader = DataLoader(trainset, batch_size=batch_size,shuffle=True)
testloader = DataLoader(testset, batch_size=batch_size,shuffle=False)

train_full = DataLoader(trainset, batch_size=len(trainset),shuffle=True)
train_X, train_y = next(iter(train_full))
train_X, train_y = train_X.to(device),train_y.to(device)
train_y=torch.tensor([class_mapping[label.item()] for label in train_y]).to(device)

test_full = DataLoader(testset, batch_size=len(testset),shuffle=True)
test_X, test_y = next(iter(test_full))
test_X, test_y = test_X.to(device),test_y.to(device)
test_y=torch.tensor([class_mapping[label.item()] for label in test_y]).to(device)

2. creat model

In [4]:
class Net(nn.Module):
    def __init__(self,inputdim):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(inputdim,512)
        self.fc2 = nn.Linear(512,3)

    def forward(self,x):
        x = torch.flatten(x,1)
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return x

net = Net(3*32*32).to(device)

3. Optimizer and loss

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(),lr = 1e-3, momentum=0.9)
max_epochs = 10

train_acc_best = 0
test_acc_best = 0
PATH="/model.pt"
for epoch in range(max_epochs):

    for i,data in enumerate(trainloader):
        optimizer.zero_grad()
        X,y = data
        X = X.to(device)
        y = y.to(device)
        y_mapped = torch.tensor([class_mapping[label.item()] for label in y]).to(device)
        # forward
        outputs = net(X)
        loss = criterion(outputs,y_mapped)
        #backward
        loss.backward()
        optimizer.step()

    with torch.no_grad():
      train_pred = net(train_X)
      train_correct = (train_pred.argmax(dim=1)==train_y).sum()
      train_acc = train_correct / len(train_pred)

      test_pred = net(test_X)
      test_correct = (test_pred.argmax(dim=1)==test_y).sum()
      test_acc = test_correct / len(test_pred)
      if test_acc_best<test_acc:
        test_acc_best = test_acc
        torch.save(net.state_dict(),PATH)


    print(f"Epoch {epoch+1}/{max_epochs}, Train accuracy: {100*train_acc:.2f}%")
    print(f"Epoch {epoch+1}/{max_epochs}, Test accuracy: {100*test_acc:.2f}%\n")

Epoch 1/10, Train accuracy: 82.97%
Epoch 1/10, Test accuracy: 80.00%

Epoch 2/10, Train accuracy: 85.62%
Epoch 2/10, Test accuracy: 81.40%

Epoch 3/10, Train accuracy: 89.49%
Epoch 3/10, Test accuracy: 83.13%

Epoch 4/10, Train accuracy: 82.69%
Epoch 4/10, Test accuracy: 76.37%

Epoch 5/10, Train accuracy: 91.15%
Epoch 5/10, Test accuracy: 83.63%

Epoch 6/10, Train accuracy: 92.31%
Epoch 6/10, Test accuracy: 83.53%

Epoch 7/10, Train accuracy: 90.69%
Epoch 7/10, Test accuracy: 81.30%

Epoch 8/10, Train accuracy: 93.15%
Epoch 8/10, Test accuracy: 83.57%

Epoch 9/10, Train accuracy: 94.41%
Epoch 9/10, Test accuracy: 84.17%

Epoch 10/10, Train accuracy: 93.81%
Epoch 10/10, Test accuracy: 83.03%



In [9]:
classes = ['car', 'dog', 'ship']
net_best = Net(3*32*32)
net_best.load_state_dict(torch.load(PATH))

correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
  for data in testloader:
    images, labels = data
    labels = torch.tensor([class_mapping[label.item()] for label in labels]).to(device)
    test_outputs = net_best(images)
    _, test_preds = torch.max(test_outputs,1)
    for label,test_pred in zip(labels,test_preds):
      if label==test_pred:
        correct_pred[classes[label]]+=1
      total_pred[classes[label]]+=1

total_acc = 0
for classname, correct_count in correct_pred.items():
  acc = 100*float(correct_count)/total_pred[classname]
  total_acc+=acc
  print(f"Accuracy for class:{classname:5s} is {acc:.2f} %")

print(f"The overall test accuracy is {total_acc/3:.2f} %")


Accuracy for class:car   is 82.40 %
Accuracy for class:dog   is 89.70 %
Accuracy for class:ship  is 80.40 %
The overall test accuracy is 84.17 %


On class dog the classifier performance best. The reason may be that the car dataset have more sufficient diversity for the classifier to distinguish features"
